### Parallel commputing in Julia
#### References
-  [A brief introduction to parallel computing in Julia](https://codingclubuc3m.github.io/2018-06-06-Parallel-computing-Julia.htmlhttps://codingclubuc3m.github.io/2018-06-06-Parallel-computing-Julia.html)
-  [Parallel Computing](https://docs.julialang.org/en/v1.4/manual/parallel-computing/)

In [1]:
using Distributed
using Base.Threads

In [2]:
addprocs(2)

2-element Vector{Int64}:
 2
 3

In [3]:
@distributed for N in 1:10
    println("The N of this iteration in $N")
end

Task (runnable) @0x00007f27f163bb20

In [4]:
# for i in workers()
#     t = rmprocs(i, waitfor=0)
#     wait(t)
    
# addprocs(4)

In [5]:
workers()

2-element Vector{Int64}:
 2
 3

In [6]:
# interrupt()

In [7]:
workers()

2-element Vector{Int64}:
 2
 3

In [8]:
@everywhere function showid()
    println("my id = ", myid())    
end

showid()

      From worker 2:	The N of this iteration in 1


      From worker 3:	The N of this iteration in 6


      From worker 3:	The N of this iteration in 7
      From worker 3:	The N of this iteration in 8


      From worker 3:	The N of this iteration in 9
      From worker 3:	The N of this iteration in 10
my id = 1
      From worker 2:	The N of this iteration in 2


      From worker 2:	The N of this iteration in 3
      From worker 2:	The N of this iteration in 4
      From worker 2:	The N of this iteration in 5


In [9]:
@everywhere showid()

my id = 1
      From worker 3:	my id = 3


      From worker 2:	my id = 2


In [10]:
Threads.@threads for N in 1:10
    println("The number of this iteration is $N")
end

The number of this iteration is 4
The number of this iteration is 1
The number of this iteration is 2
The number of this iteration is 3
The number of this iteration is 7
The number of this iteration is 5
The number of this iteration is 6
The number of this iteration is 9
The number of this iteration is 10
The number of this iteration is 8


In [11]:
# using Base.Threads

In [12]:
nthreads()

4

In [13]:
a = zeros(Int64, 10);

In [14]:
@threads for i = 1:10
   a[i] = threadid()
end

In [15]:
a

10-element Vector{Int64}:
 1
 1
 1
 2
 2
 2
 3
 3
 4
 4

### Atomic Operations
Julia supports accessing and modifying values atomically, that is, in a thread-safe way to avoid race conditions. A value (which must be of a primitive type) can be wrapped as Threads.Atomic to indicate it must be accessed in this way. Here we can see an example:

In [16]:
i = Atomic{Int}(0)

Atomic{Int64}(0)

In [17]:
ids = zeros(4);
old_is = zeros(4);

In [18]:
@threads for id in 1:4
    old_is[id] = atomic_add!(i, id)
    ids[id] = id
end

In [19]:
println(old_is)
println(ids)

[0.0, 1.0, 3.0, 6.0]
[1.0, 2.0, 3.0, 4.0]


### Side effects and mutable function arguments

In [20]:
function f()
    s = repeat(["123", "213", "231"], outer=1000)
    x = similar(s, Int)
    rx = r"1"
    @threads for i in 1:3000
        x[i] = findfirst(rx, s[i]).start
    end
    count(v -> v == 1, x)
end

f (generic function with 1 method)

In [21]:
f()

1000

In [22]:
a = zeros(100000)
@distributed for i = 1:100000
    a[i] = i
end

Task (runnable) @0x00007f27f3539a50

Filling an array

In [23]:
nthreads()

n = Int64(1e7)
a = zeros(n);
@time for i in 1:n
    a[i] = log10(i)
end

using Base.Threads
@time @threads for i in 1:n
    a[i] = log10(i)
end


  2.538598 seconds (40.00 M allocations: 763.095 MiB, 11.01% gc time, 0.68% compilation time)


  0.206157 seconds (20.02 M allocations: 306.517 MiB, 16.00% gc time, 1.63% compilation time)


[Parallel Julia webinar](https://www.youtube.com/watch?v=2SafLn0xJKY)

In [24]:
# is not thread safe
total = 0
@time @threads for i in Int(9e9)
    global total += i
end 

println("Total = ", total)

  0.019355 seconds (19.10 k allocations: 1.152 MiB, 98.70% compilation time)
Total = 9000000000


In [25]:
# is thread safe, slower

total = Atomic{Int64}(0)
@time @threads for i in Int(9e9)
    atomic_add!(total, i)
end
println("Total = ", total[])

  0.020082 seconds (18.44 k allocations: 1.105 MiB, 98.93% compilation time)
Total = 9000000000


In [26]:
function quick(n)
    total = Atomic{Int64}(0)
    @time @threads for i in 1:n
        atomic_add!(total, i)
    end
    return (total)
end

quick (generic function with 1 method)

In [27]:
quick(10)

  0.022274 seconds (16.80 k allocations: 992.424 KiB, 99.36% compilation time)


Atomic{Int64}(55)

In [28]:
quick(Int(1e8))